In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

### 1
Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X∼N(\mu,\sigma^2$) лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.


In [14]:
round(stats.norm.ppf(1- (1 - 0.997)/2), 4)

2.9677

### 2
Пусть $X∼N(\mu,\sigma^2$). Какое распределение имеет величина $\large \frac{\overline{X_n} - \mu}{S_n / \sqrt{n}}$?


- $N\left(0,1\right)$


### 3
Выберите все распределения с несимметричной функцией плотности


- хи-квадрат + 


- Гаусса -


- Фишера


- Стьюдента -



### 4
Какое из выражений задаёт доверительный интервал для разности долей в связанных выборках?


$$\hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

### 5
В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [68]:
n_1, n_2 = 11037, 11034
c_1,  c_2 = 104, 189

p_1 = c_1 / n_1
p_2 = c_2 / n_2

print(round(p_2 - p_1, 4), p_1, p_2)

0.0077 0.009422850412249705 0.017128874388254486


### 6 
   Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

$$\hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [35]:
make_beaty = lambda res: list(map(lambda x: round(x, 4), res))

In [33]:
def proportions_confint_diff_ind(p1, n1, p2, n2, alpha = 0.05): 
    import scipy.stats as sts
    import numpy as np
    
    z = sts.norm.ppf(1 - alpha / 2)   
    addit = z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    res = [(p1 - p2) + val * addit for val in (-1, 1)]
    
    return res

In [37]:
make_beaty(proportions_confint_diff_ind(p_2, n_2, p_1, n_1))

[0.0047, 0.0107]

### 7 
Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как
$$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174 $$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [38]:
def odds(p):
    return p / (1 - p)

In [46]:
make_beaty([odds(p_2)/odds(p_1)])[0]

1.8321

---
---
### 8
Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:


 - составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
 
 
 - установите random seed=0;


 - сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [61]:
def get_bootstrap_samples(data, n_samples=1000):
    np.random.seed(0)
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [92]:
def stat_intervals(stat, alpha=0.05):
    boundaries = np.percentile(
        stat,
        [100 * alpha / 2, 100 * (1 - alpha / 2)]
    )
    return boundaries

In [78]:
# 1 - asp, 2 - placebo
v_2, v_1  = [np.hstack((np.ones(c), np.zeros(n - c)))
                for (n, c) in ((n_2, c_2), (n_1, c_1))]

In [82]:
np.random.seed(0)
s1, s2 = [get_bootstrap_samples(v) for v in (v_1, v_2)]

In [89]:
boots = [np.sum(b) / b.shape[0] / (np.sum(a) / a.shape[0])
             for a, b in zip(s1, s2)
        ]

In [94]:
make_beaty(stat_intervals(boots))

[1.6199, 2.0841]